In [1]:
# Initialization
!pip install playwright
!playwright install

from playwright.async_api import async_playwright
import re

In [34]:
async def scrape(url):
    async with async_playwright() as playwright:

        # Initialization
        browser = await playwright.chromium.launch()
        page = await browser.new_page()

        # Input the desired Amazon url 
        await page.goto(url)
        
        # find the dollar amount price, while removing whitespace using filter() 
        await page.wait_for_selector('.a-price-whole')
        dollar = await page.inner_text('.a-price-whole')
        dollar = ''.join(filter(str.isdigit, str(dollar))) # filter for digits only

        # find the fraction price
        await page.wait_for_selector('.a-price-fraction')
        fraction = await page.inner_text('.a-price-fraction')

        # combine the dollar amount and fraction together
        price = dollar + '.' + fraction

        # convert the string into a float
        price = float(price)
        
        await browser.close()
        return price
        

# Input the list of urls which act as a wishlist
wishlist = ['https://www.amazon.ca/dp/B07GBZ4Q68',
            'https://www.amazon.ca/dp/B0CL7RQRY9',
            'https://www.amazon.ca/dp/B0BJLXTYMN']

# Initialization of price
price = 0
tax_rate = 1.13 # enter your local tax rate HST/GST

# loops through each element in wishlist, using the each url in the scraping algorithm
for i in range(0,len(wishlist)):
    price += await scrape(wishlist[i])

# finalize by factoring in tax and rounding
price = round(price * tax_rate, 2)
    
# Output price
print('The total price of the wishlist is: $', price, sep='')

The total price of the wishlist is: $1174.01
